In [1]:
from statsmodels.tsa.stattools import grangercausalitytests
import pandas as pd
import numpy as np

In [2]:
# 修改日期格式
def date_format(dt):
    dt = str(dt)
    year = dt[:dt.find("年")]
    month = dt[dt.find("年")+1:dt.find("月")]
    day = dt[dt.find("月")+1:dt.find("日")]
    if len(month) == 1:
        month = '0' + month
    if len(day) == 1:
        day = '0' + day
    return year + '-' + month + '-' + day

# Top100

In [3]:
df = pd.read_excel("../相关性系数/日维度关键词热度排序总表.xlsx")
df = df.pivot_table(index='日期',columns='关键词',values='系数',aggfunc='sum').fillna(0)
df2 = pd.read_excel("../相关性系数/相关系数排名-中美汇率.xlsx")
df = df[list(df2['关键词'][:100])]
df

关键词,美国,口罩,武汉,湖北,开学,新冠,礼裙,原来,钟南山,肺炎,...,山河,冰糖,物资,刘雨昕,三叉戟,传闻中,分享,周放,确诊,长津湖
日期,,,,,,,,,,,,,,,,,,,,,
2020-01-01,0.000000,0.0,0.000301,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
2020-01-02,0.000000,0.0,0.011621,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.022819,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
2020-01-03,0.010290,0.0,0.008366,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.008366,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000798,0.0,0.000000,0.000000
2020-01-04,0.033193,0.0,0.002468,0.0,0.0,0.000000,0.000000,0.009067,0.000000,0.002468,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
2020-01-05,0.017324,0.0,0.022405,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.022405,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000252,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,0.007717,0.0,0.000000,0.0,0.0,0.005551,0.000000,0.008998,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.028130,0.000000
2021-12-28,0.015005,0.0,0.003299,0.0,0.0,0.009740,0.000936,0.004694,0.000000,0.000000,...,0.0,0.0,0.004234,0.0,0.0,0.0,0.016253,0.0,0.023056,0.001712
2021-12-29,0.006471,0.0,0.000000,0.0,0.0,0.000398,0.000000,0.011634,0.002312,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.002807,0.0,0.019321,0.000000


In [4]:
df1 = pd.read_csv("../相关性系数/USD_CNY历史数据.csv")
df1['日期'] = df1['日期'].apply(date_format)
df1 = df1.sort_values(by='日期').set_index('日期')
df1 = df1['收盘']
df3 = pd.merge(df1, df, how='left',left_index=True,right_index=True)

In [5]:
res_table = pd.DataFrame()
maxlag = 2  # lag值
for column_name in df3.columns[1:]:
    sig = []
    for i in range(maxlag):
        pvalue = grangercausalitytests(df3[['收盘', column_name]], maxlag=maxlag)[i+1][0][ 'params_ftest'][1]
        if pvalue < 0.05:
            sig.append(1)
        else:
            sig.append(0)
        
    res_table[column_name] = sig



Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.2092  , p=0.6476  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.2104  , p=0.6465  , df=1
likelihood ratio test: chi2=0.2104  , p=0.6465  , df=1
parameter F test:         F=0.2092  , p=0.6476  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.1059  , p=0.8995  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=0.2139  , p=0.8986  , df=2
likelihood ratio test: chi2=0.2139  , p=0.8986  , df=2
parameter F test:         F=0.1059  , p=0.8995  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.2092  , p=0.6476  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.2104  , p=0.6465  , df=1
likelihood ratio test: chi2=0.2104  , p=0.6465  , df=1
parameter F test:         F=0.2092  , p=0.6476  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.1059  , p=0.8995  

likelihood ratio test: chi2=0.4037  , p=0.5252  , df=1
parameter F test:         F=0.4015  , p=0.5266  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.3753  , p=0.6873  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=0.7582  , p=0.6845  , df=2
likelihood ratio test: chi2=0.7576  , p=0.6847  , df=2
parameter F test:         F=0.3753  , p=0.6873  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0154  , p=0.9012  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.0155  , p=0.9008  , df=1
likelihood ratio test: chi2=0.0155  , p=0.9008  , df=1
parameter F test:         F=0.0154  , p=0.9012  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=2.0427  , p=0.1308  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=4.1262  , p=0.1271  , df=2
likelihood ratio test: chi2=4.1094  , p=0.1281  , df=2
parameter F test:         F=2.0427  , p

parameter F test:         F=2.4391  , p=0.0883  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=2.7005  , p=0.1009  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=2.7166  , p=0.0993  , df=1
likelihood ratio test: chi2=2.7094  , p=0.0998  , df=1
parameter F test:         F=2.7005  , p=0.1009  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.4419  , p=0.2375  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=2.9127  , p=0.2331  , df=2
likelihood ratio test: chi2=2.9043  , p=0.2341  , df=2
parameter F test:         F=1.4419  , p=0.2375  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=2.7005  , p=0.1009  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=2.7166  , p=0.0993  , df=1
likelihood ratio test: chi2=2.7094  , p=0.0998  , df=1
parameter F test:         F=2.7005  , p=0.1009  , df_denom=503, df_num=1

Granger Causality
nu

ssr based F test:         F=0.0301  , p=0.8624  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.0302  , p=0.8620  , df=1
likelihood ratio test: chi2=0.0302  , p=0.8620  , df=1
parameter F test:         F=0.0301  , p=0.8624  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.2604  , p=0.7709  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=0.5260  , p=0.7688  , df=2
likelihood ratio test: chi2=0.5257  , p=0.7689  , df=2
parameter F test:         F=0.2604  , p=0.7709  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1294  , p=0.7192  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.1302  , p=0.7182  , df=1
likelihood ratio test: chi2=0.1302  , p=0.7183  , df=1
parameter F test:         F=0.1294  , p=0.7192  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=4.7629  , p=0.0089  , df_denom=500, df_num=2
ssr based chi2 test: 

parameter F test:         F=1.1680  , p=0.3118  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0136  , p=0.9071  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.0137  , p=0.9068  , df=1
likelihood ratio test: chi2=0.0137  , p=0.9068  , df=1
parameter F test:         F=0.0136  , p=0.9071  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.1680  , p=0.3118  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=2.3593  , p=0.3074  , df=2
likelihood ratio test: chi2=2.3538  , p=0.3082  , df=2
parameter F test:         F=1.1680  , p=0.3118  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=3.6830  , p=0.0555  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=3.7049  , p=0.0543  , df=1
likelihood ratio test: chi2=3.6914  , p=0.0547  , df=1
parameter F test:         F=3.6830  , p=0.0555  , df_denom=503, df_num=1

Granger Causality
nu

...

ssr based F test:         F=13.9733 , p=0.0002  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=14.0566 , p=0.0002  , df=1
likelihood ratio test: chi2=13.8649 , p=0.0002  , df=1
parameter F test:         F=13.9733 , p=0.0002  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=7.2809  , p=0.0008  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=14.7074 , p=0.0006  , df=2
likelihood ratio test: chi2=14.4973 , p=0.0007  , df=2
parameter F test:         F=7.2809  , p=0.0008  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=13.9733 , p=0.0002  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=14.0566 , p=0.0002  , df=1
likelihood ratio test: chi2=13.8649 , p=0.0002  , df=1
parameter F test:         F=13.9733 , p=0.0002  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=7.2809  , p=0.0008  , df_denom=500, df_num=2
ssr based chi2 test: 


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0136  , p=0.9072  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.0137  , p=0.9068  , df=1
likelihood ratio test: chi2=0.0137  , p=0.9068  , df=1
parameter F test:         F=0.0136  , p=0.9072  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.0647  , p=0.9373  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=0.1307  , p=0.9367  , df=2
likelihood ratio test: chi2=0.1307  , p=0.9367  , df=2
parameter F test:         F=0.0647  , p=0.9373  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.0836  , p=0.2984  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=1.0901  , p=0.2965  , df=1
likelihood ratio test: chi2=1.0889  , p=0.2967  , df=1
parameter F test:         F=1.0836  , p=0.2984  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.5656  , p=0.5684  

likelihood ratio test: chi2=0.9258  , p=0.6295  , df=2
parameter F test:         F=0.4587  , p=0.6323  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=7.7508  , p=0.0056  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=7.7971  , p=0.0052  , df=1
likelihood ratio test: chi2=7.7376  , p=0.0054  , df=1
parameter F test:         F=7.7508  , p=0.0056  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=4.9629  , p=0.0073  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=10.0251 , p=0.0067  , df=2
likelihood ratio test: chi2=9.9269  , p=0.0070  , df=2
parameter F test:         F=4.9629  , p=0.0073  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=7.7508  , p=0.0056  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=7.7971  , p=0.0052  , df=1
likelihood ratio test: chi2=7.7376  , p=0.0054  , df=1
parameter F test:         F=7.7508  , p

likelihood ratio test: chi2=0.8193  , p=0.3654  , df=1
parameter F test:         F=0.8151  , p=0.3671  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.4819  , p=0.6179  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=0.9734  , p=0.6146  , df=2
likelihood ratio test: chi2=0.9725  , p=0.6149  , df=2
parameter F test:         F=0.4819  , p=0.6179  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.8151  , p=0.3671  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.8199  , p=0.3652  , df=1
likelihood ratio test: chi2=0.8193  , p=0.3654  , df=1
parameter F test:         F=0.8151  , p=0.3671  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.4819  , p=0.6179  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=0.9734  , p=0.6146  , df=2
likelihood ratio test: chi2=0.9725  , p=0.6149  , df=2
parameter F test:         F=0.4819  , p

likelihood ratio test: chi2=0.0121  , p=0.9124  , df=1
parameter F test:         F=0.0120  , p=0.9127  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.4409  , p=0.6437  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=0.8905  , p=0.6407  , df=2
likelihood ratio test: chi2=0.8897  , p=0.6409  , df=2
parameter F test:         F=0.4409  , p=0.6437  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0788  , p=0.7790  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.0793  , p=0.7783  , df=1
likelihood ratio test: chi2=0.0793  , p=0.7783  , df=1
parameter F test:         F=0.0788  , p=0.7790  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.0870  , p=0.9167  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=0.1757  , p=0.9159  , df=2
likelihood ratio test: chi2=0.1757  , p=0.9159  , df=2
parameter F test:         F=0.0870  , p

In [6]:
pd.set_option('display.max_columns',0) #replace n with the number of columns you want to see completely
pd.set_option('display.max_rows',0) #replace n with the number of rows you want to see completely

res_table.T.to_excel('中美汇率Top100-正向.xlsx')
res_table

,美国,口罩,武汉,湖北,开学,新冠,礼裙,原来,钟南山,肺炎,病例,本土,接种,出院,意大利,抗疫,特朗普,护士,世卫,疫苗,破防,感染,患者,无价,万例,时间,李兰娟,之姐,输入,推迟,女子,2021,复课,乘风破浪,号房,境外,上网,纽约州,纽约,方舱,...,风起,毕业生,医疗队,大赛,医护人员,宋凛,留学生,开播,死亡,例新冠,称新冠,洛阳,重症,如何,高三,龚俊,本轮,变异,猫咪,医护,医院,李宇春,六中全会,呼吸机,美股,宋亚轩,网课,作人,出现,上海,山河,冰糖,物资,刘雨昕,三叉戟,传闻中,分享,周放,确诊,长津湖
0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
1,0,1,1,1,0,0,0,0,0,0,1,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0


## 反向

In [7]:
res_table = pd.DataFrame()
maxlag = 2  # lag值
for column_name in df3.columns[1:]:
    sig = []
    for i in range(maxlag):
        pvalue = grangercausalitytests(df3[[column_name,'收盘']], maxlag=maxlag)[i+1][0][ 'params_ftest'][1]
        if pvalue < 0.05:
            sig.append(1)
        else:
            sig.append(0)
        
    res_table[column_name] = sig



Granger Causality
number of lags (no zero) 1
ssr based F test:         F=38.6815 , p=0.0000  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=38.9122 , p=0.0000  , df=1
likelihood ratio test: chi2=37.4885 , p=0.0000  , df=1
parameter F test:         F=38.6815 , p=0.0000  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=14.8537 , p=0.0000  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=30.0045 , p=0.0000  , df=2
likelihood ratio test: chi2=29.1469 , p=0.0000  , df=2
parameter F test:         F=14.8537 , p=0.0000  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=38.6815 , p=0.0000  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=38.9122 , p=0.0000  , df=1
likelihood ratio test: chi2=37.4885 , p=0.0000  , df=1
parameter F test:         F=38.6815 , p=0.0000  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=14.8537 , p=0.0000  

likelihood ratio test: chi2=33.9354 , p=0.0000  , df=1
parameter F test:         F=34.8911 , p=0.0000  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=16.6156 , p=0.0000  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=33.5636 , p=0.0000  , df=2
likelihood ratio test: chi2=32.4953 , p=0.0000  , df=2
parameter F test:         F=16.6156 , p=0.0000  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=33.1410 , p=0.0000  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=33.3387 , p=0.0000  , df=1
likelihood ratio test: chi2=32.2864 , p=0.0000  , df=1
parameter F test:         F=33.1410 , p=0.0000  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=13.3430 , p=0.0000  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=26.9529 , p=0.0000  , df=2
likelihood ratio test: chi2=26.2583 , p=0.0000  , df=2
parameter F test:         F=13.3430 , p

ssr based F test:         F=13.4070 , p=0.0003  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=13.4869 , p=0.0002  , df=1
likelihood ratio test: chi2=13.3103 , p=0.0003  , df=1
parameter F test:         F=13.4070 , p=0.0003  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=3.9814  , p=0.0193  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=8.0424  , p=0.0179  , df=2
likelihood ratio test: chi2=7.9790  , p=0.0185  , df=2
parameter F test:         F=3.9814  , p=0.0193  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=27.7143 , p=0.0000  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=27.8796 , p=0.0000  , df=1
likelihood ratio test: chi2=27.1386 , p=0.0000  , df=1
parameter F test:         F=27.7143 , p=0.0000  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=15.8371 , p=0.0000  , df_denom=500, df_num=2
ssr based chi2 test: 

likelihood ratio test: chi2=9.2345  , p=0.0099  , df=2
parameter F test:         F=4.6136  , p=0.0103  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=15.3845 , p=0.0001  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=15.4763 , p=0.0001  , df=1
likelihood ratio test: chi2=15.2443 , p=0.0001  , df=1
parameter F test:         F=15.3845 , p=0.0001  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=6.7598  , p=0.0013  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=13.6548 , p=0.0011  , df=2
likelihood ratio test: chi2=13.4734 , p=0.0012  , df=2
parameter F test:         F=6.7598  , p=0.0013  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=15.3845 , p=0.0001  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=15.4763 , p=0.0001  , df=1
likelihood ratio test: chi2=15.2443 , p=0.0001  , df=1
parameter F test:         F=15.3845 , p

parameter F test:         F=21.6350 , p=0.0000  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=7.0949  , p=0.0009  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=14.3317 , p=0.0008  , df=2
likelihood ratio test: chi2=14.1321 , p=0.0009  , df=2
parameter F test:         F=7.0949  , p=0.0009  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=21.6350 , p=0.0000  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=21.7640 , p=0.0000  , df=1
likelihood ratio test: chi2=21.3090 , p=0.0000  , df=1
parameter F test:         F=21.6350 , p=0.0000  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=7.0949  , p=0.0009  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=14.3317 , p=0.0008  , df=2
likelihood ratio test: chi2=14.1321 , p=0.0009  , df=2
parameter F test:         F=7.0949  , p=0.0009  , df_denom=500, df_num=2

Granger Causality
nu

...

parameter F test:         F=13.2389 , p=0.0003  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=4.6128  , p=0.0103  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=9.3179  , p=0.0095  , df=2
likelihood ratio test: chi2=9.2330  , p=0.0099  , df=2
parameter F test:         F=4.6128  , p=0.0103  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=13.2389 , p=0.0003  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=13.3179 , p=0.0003  , df=1
likelihood ratio test: chi2=13.1456 , p=0.0003  , df=1
parameter F test:         F=13.2389 , p=0.0003  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=4.6128  , p=0.0103  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=9.3179  , p=0.0095  , df=2
likelihood ratio test: chi2=9.2330  , p=0.0099  , df=2
parameter F test:         F=4.6128  , p=0.0103  , df_denom=500, df_num=2

Granger Causality
nu

ssr based F test:         F=2.3156  , p=0.0998  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=4.6774  , p=0.0965  , df=2
likelihood ratio test: chi2=4.6559  , p=0.0975  , df=2
parameter F test:         F=2.3156  , p=0.0998  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=25.9607 , p=0.0000  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=26.1155 , p=0.0000  , df=1
likelihood ratio test: chi2=25.4639 , p=0.0000  , df=1
parameter F test:         F=25.9607 , p=0.0000  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=14.1006 , p=0.0000  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=28.4831 , p=0.0000  , df=2
likelihood ratio test: chi2=27.7089 , p=0.0000  , df=2
parameter F test:         F=14.1006 , p=0.0000  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=25.9607 , p=0.0000  , df_denom=503, df_num=1
ssr based chi2 test: 

ssr based F test:         F=14.3067 , p=0.0002  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=14.3920 , p=0.0001  , df=1
likelihood ratio test: chi2=14.1912 , p=0.0002  , df=1
parameter F test:         F=14.3067 , p=0.0002  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=5.8727  , p=0.0030  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=11.8629 , p=0.0027  , df=2
likelihood ratio test: chi2=11.7258 , p=0.0028  , df=2
parameter F test:         F=5.8727  , p=0.0030  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=10.7355 , p=0.0011  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=10.7996 , p=0.0010  , df=1
likelihood ratio test: chi2=10.6859 , p=0.0011  , df=1
parameter F test:         F=10.7355 , p=0.0011  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=5.1461  , p=0.0061  , df_denom=500, df_num=2
ssr based chi2 test: 

ssr based F test:         F=1.1741  , p=0.3099  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=2.3717  , p=0.3055  , df=2
likelihood ratio test: chi2=2.3661  , p=0.3063  , df=2
parameter F test:         F=1.1741  , p=0.3099  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=11.5527 , p=0.0007  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=11.6216 , p=0.0007  , df=1
likelihood ratio test: chi2=11.4902 , p=0.0007  , df=1
parameter F test:         F=11.5527 , p=0.0007  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=3.3440  , p=0.0361  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=6.7548  , p=0.0341  , df=2
likelihood ratio test: chi2=6.7101  , p=0.0349  , df=2
parameter F test:         F=3.3440  , p=0.0361  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=11.5527 , p=0.0007  , df_denom=503, df_num=1
ssr based chi2 test: 


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=3.7466  , p=0.0535  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=3.7690  , p=0.0522  , df=1
likelihood ratio test: chi2=3.7550  , p=0.0526  , df=1
parameter F test:         F=3.7466  , p=0.0535  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.6985  , p=0.1840  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=3.4309  , p=0.1799  , df=2
likelihood ratio test: chi2=3.4193  , p=0.1809  , df=2
parameter F test:         F=1.6985  , p=0.1840  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=3.7466  , p=0.0535  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=3.7690  , p=0.0522  , df=1
likelihood ratio test: chi2=3.7550  , p=0.0526  , df=1
parameter F test:         F=3.7466  , p=0.0535  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.6985  , p=0.1840  

In [8]:
pd.set_option('display.max_columns',0) #replace n with the number of columns you want to see completely
pd.set_option('display.max_rows',0) #replace n with the number of rows you want to see completely

res_table.T.to_excel('中美汇率Top100-反向.xlsx')
res_table

,美国,口罩,武汉,湖北,开学,新冠,礼裙,原来,钟南山,肺炎,病例,本土,接种,出院,意大利,抗疫,特朗普,护士,世卫,疫苗,破防,感染,患者,无价,万例,时间,李兰娟,之姐,输入,推迟,女子,2021,复课,乘风破浪,号房,境外,上网,纽约州,纽约,方舱,...,风起,毕业生,医疗队,大赛,医护人员,宋凛,留学生,开播,死亡,例新冠,称新冠,洛阳,重症,如何,高三,龚俊,本轮,变异,猫咪,医护,医院,李宇春,六中全会,呼吸机,美股,宋亚轩,网课,作人,出现,上海,山河,冰糖,物资,刘雨昕,三叉戟,传闻中,分享,周放,确诊,长津湖
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1
1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,0,1,0,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,0,0


# Top500

In [10]:
df = pd.read_excel("../相关性系数/日维度关键词热度排序总表.xlsx")
df = df.pivot_table(index='日期',columns='关键词',values='系数',aggfunc='sum').fillna(0)
df2 = pd.read_excel("../相关性系数/相关系数排名-中美汇率.xlsx")
df = df[list(df2['关键词'][:500])]
df

关键词,美国,口罩,武汉,湖北,开学,新冠,礼裙,原来,钟南山,肺炎,病例,本土,接种,出院,意大利,抗疫,特朗普,护士,世卫,疫苗,破防,感染,患者,无价,万例,时间,李兰娟,之姐,输入,推迟,女子,2021,复课,乘风破浪,号房,境外,上网,纽约州,纽约,方舱,...,海市,各级,淡黄,顾佳,20,无能,受难,英国,挑战,汉克斯,航天员,空间站,马嘉祺,当家,全运会,不明,台湾,白宫,夜店,白鹿,主母,秦墩,半夜,华盛顿,好汉歌,淮南,吴宣仪,深夜,定金,变装,场面,忽悠,孝感,套装,柳絮,下药,贝宁,庆怜,最甜,王者
日期,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01,0.000000,0.000000,0.000301,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.029700,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.029902,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.002515,0.000000
2020-01-02,0.000000,0.000000,0.011621,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.022819,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001084,0.0,0.000000,0.000000,0.000000,0.024331,0.0,0.000000,0.011259,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.016160,0.012929,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2020-01-03,0.010290,0.000000,0.008366,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.008366,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001766,0.0,0.044838,0.000000,0.000000,0.001422,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00084,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.001930,0.0,0.0,0.002408,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.007525,0.000385,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.002290
2020-01-04,0.033193,0.000000,0.002468,0.0,0.0,0.000000,0.000000,0.009067,0.000000,0.002468,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006097,0.000000,0.0,0.009777,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.004371,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.002468,0.001906,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.022532,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000463
2020-01-05,0.017324,0.000000,0.022405,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.022405,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.022405,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.001162
2020-01-06,0.016815,0.000000,0.012669,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.005832,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011100,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.006913,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000338,0.0,0.0,0.000000,0.0,0.001527,0.000000,0.000000,0.00000,0.000000,0.0,0.005832,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000760
2020-01-07,0.037970,0.027081,0.002154,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024339

In [10]:
df1 = pd.read_csv("../相关性系数/USD_CNY历史数据.csv")
df1['日期'] = df1['日期'].apply(date_format)
df1 = df1.sort_values(by='日期').set_index('日期')
df1 = df1['收盘']
df3 = pd.merge(df1, df, how='left',left_index=True,right_index=True)

In [11]:
res_table = pd.DataFrame()
maxlag = 2  # lag值
for column_name in df3.columns[1:]:
    sig = []
    for i in range(maxlag):
        pvalue = grangercausalitytests(df3[['收盘', column_name]], maxlag=maxlag)[i+1][0][ 'params_ftest'][1]
        if pvalue < 0.05:
            sig.append(1)
        else:
            sig.append(0)
        
    res_table[column_name] = sig



Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.2092  , p=0.6476  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.2104  , p=0.6465  , df=1
likelihood ratio test: chi2=0.2104  , p=0.6465  , df=1
parameter F test:         F=0.2092  , p=0.6476  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.1059  , p=0.8995  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=0.2139  , p=0.8986  , df=2
likelihood ratio test: chi2=0.2139  , p=0.8986  , df=2
parameter F test:         F=0.1059  , p=0.8995  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.2092  , p=0.6476  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.2104  , p=0.6465  , df=1
likelihood ratio test: chi2=0.2104  , p=0.6465  , df=1
parameter F test:         F=0.2092  , p=0.6476  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.1059  , p=0.8995  

ssr based F test:         F=2.0216  , p=0.1335  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=4.0837  , p=0.1298  , df=2
likelihood ratio test: chi2=4.0673  , p=0.1309  , df=2
parameter F test:         F=2.0216  , p=0.1335  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=2.5629  , p=0.1100  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=2.5782  , p=0.1083  , df=1
likelihood ratio test: chi2=2.5717  , p=0.1088  , df=1
parameter F test:         F=2.5629  , p=0.1100  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=2.0216  , p=0.1335  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=4.0837  , p=0.1298  , df=2
likelihood ratio test: chi2=4.0673  , p=0.1309  , df=2
parameter F test:         F=2.0216  , p=0.1335  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=8.9263  , p=0.0029  , df_denom=503, df_num=1
ssr based chi2 test: 

number of lags (no zero) 1
ssr based F test:         F=0.2108  , p=0.6464  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.2120  , p=0.6452  , df=1
likelihood ratio test: chi2=0.2120  , p=0.6452  , df=1
parameter F test:         F=0.2108  , p=0.6464  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.6595  , p=0.5175  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=1.3323  , p=0.5137  , df=2
likelihood ratio test: chi2=1.3305  , p=0.5141  , df=2
parameter F test:         F=0.6595  , p=0.5175  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.2108  , p=0.6464  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.2120  , p=0.6452  , df=1
likelihood ratio test: chi2=0.2120  , p=0.6452  , df=1
parameter F test:         F=0.2108  , p=0.6464  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.6595  , p=0.5175  , df_denom=500, df_

likelihood ratio test: chi2=0.2199  , p=0.6391  , df=1
parameter F test:         F=0.2186  , p=0.6403  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.7449  , p=0.4753  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=1.5046  , p=0.4713  , df=2
likelihood ratio test: chi2=1.5024  , p=0.4718  , df=2
parameter F test:         F=0.7449  , p=0.4753  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.2186  , p=0.6403  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.2199  , p=0.6391  , df=1
likelihood ratio test: chi2=0.2199  , p=0.6391  , df=1
parameter F test:         F=0.2186  , p=0.6403  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.7449  , p=0.4753  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=1.5046  , p=0.4713  , df=2
likelihood ratio test: chi2=1.5024  , p=0.4718  , df=2
parameter F test:         F=0.7449  , p

ssr based chi2 test:   chi2=0.7199  , p=0.6977  , df=2
likelihood ratio test: chi2=0.7194  , p=0.6979  , df=2
parameter F test:         F=0.3564  , p=0.7004  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0337  , p=0.8544  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.0339  , p=0.8539  , df=1
likelihood ratio test: chi2=0.0339  , p=0.8539  , df=1
parameter F test:         F=0.0337  , p=0.8544  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.3564  , p=0.7004  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=0.7199  , p=0.6977  , df=2
likelihood ratio test: chi2=0.7194  , p=0.6979  , df=2
parameter F test:         F=0.3564  , p=0.7004  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.4044  , p=0.5251  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.4068  , p=0.5236  , df=1
likelihood ratio test: chi2=0.4067  , p

...

likelihood ratio test: chi2=1.7962  , p=0.4073  , df=2
parameter F test:         F=0.8908  , p=0.4110  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.4451  , p=0.2299  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=1.4537  , p=0.2279  , df=1
likelihood ratio test: chi2=1.4517  , p=0.2283  , df=1
parameter F test:         F=1.4451  , p=0.2299  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.8908  , p=0.4110  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=1.7994  , p=0.4067  , df=2
likelihood ratio test: chi2=1.7962  , p=0.4073  , df=2
parameter F test:         F=0.8908  , p=0.4110  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=2.0897  , p=0.1489  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=2.1022  , p=0.1471  , df=1
likelihood ratio test: chi2=2.0978  , p=0.1475  , df=1
parameter F test:         F=2.0897  , p

parameter F test:         F=0.7736  , p=0.4619  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.4957  , p=0.4817  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.4987  , p=0.4801  , df=1
likelihood ratio test: chi2=0.4984  , p=0.4802  , df=1
parameter F test:         F=0.4957  , p=0.4817  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.7736  , p=0.4619  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=1.5626  , p=0.4578  , df=2
likelihood ratio test: chi2=1.5602  , p=0.4584  , df=2
parameter F test:         F=0.7736  , p=0.4619  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1315  , p=0.7170  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.1323  , p=0.7160  , df=1
likelihood ratio test: chi2=0.1323  , p=0.7160  , df=1
parameter F test:         F=0.1315  , p=0.7170  , df_denom=503, df_num=1

Granger Causality
nu

parameter F test:         F=0.4794  , p=0.6194  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.8227  , p=0.3648  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.8276  , p=0.3630  , df=1
likelihood ratio test: chi2=0.8269  , p=0.3632  , df=1
parameter F test:         F=0.8227  , p=0.3648  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.4794  , p=0.6194  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=0.9684  , p=0.6162  , df=2
likelihood ratio test: chi2=0.9675  , p=0.6165  , df=2
parameter F test:         F=0.4794  , p=0.6194  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1931  , p=0.6605  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.1943  , p=0.6594  , df=1
likelihood ratio test: chi2=0.1943  , p=0.6594  , df=1
parameter F test:         F=0.1931  , p=0.6605  , df_denom=503, df_num=1

Granger Causality
nu

likelihood ratio test: chi2=1.6359  , p=0.4413  , df=2
parameter F test:         F=0.8112  , p=0.4449  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.4731  , p=0.2254  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=1.4819  , p=0.2235  , df=1
likelihood ratio test: chi2=1.4797  , p=0.2238  , df=1
parameter F test:         F=1.4731  , p=0.2254  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.8112  , p=0.4449  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=1.6385  , p=0.4408  , df=2
likelihood ratio test: chi2=1.6359  , p=0.4413  , df=2
parameter F test:         F=0.8112  , p=0.4449  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.3468  , p=0.2464  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=1.3548  , p=0.2444  , df=1
likelihood ratio test: chi2=1.3530  , p=0.2448  , df=1
parameter F test:         F=1.3468  , p

parameter F test:         F=0.6331  , p=0.4266  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.5271  , p=0.5906  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=1.0648  , p=0.5872  , df=2
likelihood ratio test: chi2=1.0637  , p=0.5875  , df=2
parameter F test:         F=0.5271  , p=0.5906  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.6331  , p=0.4266  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.6369  , p=0.4248  , df=1
likelihood ratio test: chi2=0.6365  , p=0.4250  , df=1
parameter F test:         F=0.6331  , p=0.4266  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.5271  , p=0.5906  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=1.0648  , p=0.5872  , df=2
likelihood ratio test: chi2=1.0637  , p=0.5875  , df=2
parameter F test:         F=0.5271  , p=0.5906  , df_denom=500, df_num=2

Granger Causality
nu

In [12]:
pd.set_option('display.max_columns',0) #replace n with the number of columns you want to see completely
pd.set_option('display.max_rows',0) #replace n with the number of rows you want to see completely

res_table.T.to_excel('中美汇率Top500-正向.xlsx')
res_table

,美国,口罩,武汉,湖北,开学,新冠,礼裙,原来,钟南山,肺炎,病例,本土,接种,出院,意大利,抗疫,特朗普,护士,世卫,疫苗,破防,感染,患者,无价,万例,时间,李兰娟,之姐,输入,推迟,女子,2021,复课,乘风破浪,号房,境外,上网,纽约州,纽约,方舱,触手可及,社死,宋仁宗,返校,男子,组织,累计,时代,非洲,医疗,援鄂,看待,重启,清平乐,复工,狗狗,自己,香港,历史,顶楼,...,东北,医务,emo,邮轮,UNINE,豆瓣,股市,明星,石家庄,不会,冬天,非裔,橘生,全球,想见,年级,义工,仙剑,54,症状,海市,各级,淡黄,顾佳,20,无能,受难,英国,挑战,汉克斯,航天员,空间站,马嘉祺,当家,全运会,不明,台湾,白宫,夜店,白鹿,主母,秦墩,半夜,华盛顿,好汉歌,淮南,吴宣仪,深夜,定金,变装,场面,忽悠,孝感,套装,柳絮,下药,贝宁,庆怜,最甜,王者
0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,1,1,0,0,0,0,0,0,1,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## 反向

In [13]:
res_table = pd.DataFrame()
maxlag = 2  # lag值
for column_name in df3.columns[1:]:
    sig = []
    for i in range(maxlag):
        pvalue = grangercausalitytests(df3[[column_name,'收盘']], maxlag=maxlag)[i+1][0][ 'params_ftest'][1]
        if pvalue < 0.05:
            sig.append(1)
        else:
            sig.append(0)
        
    res_table[column_name] = sig



Granger Causality
number of lags (no zero) 1
ssr based F test:         F=38.6815 , p=0.0000  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=38.9122 , p=0.0000  , df=1
likelihood ratio test: chi2=37.4885 , p=0.0000  , df=1
parameter F test:         F=38.6815 , p=0.0000  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=14.8537 , p=0.0000  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=30.0045 , p=0.0000  , df=2
likelihood ratio test: chi2=29.1469 , p=0.0000  , df=2
parameter F test:         F=14.8537 , p=0.0000  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=38.6815 , p=0.0000  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=38.9122 , p=0.0000  , df=1
likelihood ratio test: chi2=37.4885 , p=0.0000  , df=1
parameter F test:         F=38.6815 , p=0.0000  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=14.8537 , p=0.0000  

ssr based chi2 test:   chi2=6.8705  , p=0.0322  , df=2
likelihood ratio test: chi2=6.8242  , p=0.0330  , df=2
parameter F test:         F=3.4012  , p=0.0341  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=5.0764  , p=0.0247  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=5.1067  , p=0.0238  , df=1
likelihood ratio test: chi2=5.0811  , p=0.0242  , df=1
parameter F test:         F=5.0764  , p=0.0247  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=3.4012  , p=0.0341  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=6.8705  , p=0.0322  , df=2
likelihood ratio test: chi2=6.8242  , p=0.0330  , df=2
parameter F test:         F=3.4012  , p=0.0341  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=8.5131  , p=0.0037  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=8.5639  , p=0.0034  , df=1
likelihood ratio test: chi2=8.4922  , p

parameter F test:         F=22.5323 , p=0.0000  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=8.3346  , p=0.0003  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=16.8359 , p=0.0002  , df=2
likelihood ratio test: chi2=16.5613 , p=0.0003  , df=2
parameter F test:         F=8.3346  , p=0.0003  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=22.5323 , p=0.0000  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=22.6667 , p=0.0000  , df=1
likelihood ratio test: chi2=22.1736 , p=0.0000  , df=1
parameter F test:         F=22.5323 , p=0.0000  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=8.3346  , p=0.0003  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=16.8359 , p=0.0002  , df=2
likelihood ratio test: chi2=16.5613 , p=0.0003  , df=2
parameter F test:         F=8.3346  , p=0.0003  , df_denom=500, df_num=2

Granger Causality
nu

likelihood ratio test: chi2=13.4734 , p=0.0012  , df=2
parameter F test:         F=6.7598  , p=0.0013  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=28.1618 , p=0.0000  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=28.3298 , p=0.0000  , df=1
likelihood ratio test: chi2=27.5651 , p=0.0000  , df=1
parameter F test:         F=28.1618 , p=0.0000  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=11.5095 , p=0.0000  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=23.2492 , p=0.0000  , df=2
likelihood ratio test: chi2=22.7299 , p=0.0000  , df=2
parameter F test:         F=11.5095 , p=0.0000  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=28.1618 , p=0.0000  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=28.3298 , p=0.0000  , df=1
likelihood ratio test: chi2=27.5651 , p=0.0000  , df=1
parameter F test:         F=28.1618 , p

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=8.4067  , p=0.0039  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=8.4569  , p=0.0036  , df=1
likelihood ratio test: chi2=8.3870  , p=0.0038  , df=1
parameter F test:         F=8.4067  , p=0.0039  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=3.3821  , p=0.0348  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=6.8318  , p=0.0328  , df=2
likelihood ratio test: chi2=6.7860  , p=0.0336  , df=2
parameter F test:         F=3.3821  , p=0.0348  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=25.8500 , p=0.0000  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=26.0042 , p=0.0000  , df=1
likelihood ratio test: chi2=25.3580 , p=0.0000  , df=1
parameter F test:         F=25.8500 , p=0.0000  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=12.1884 , p=0.0000  ,

...

ssr based F test:         F=2.4833  , p=0.0845  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=5.0162  , p=0.0814  , df=2
likelihood ratio test: chi2=4.9914  , p=0.0824  , df=2
parameter F test:         F=2.4833  , p=0.0845  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=7.2930  , p=0.0072  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=7.3365  , p=0.0068  , df=1
likelihood ratio test: chi2=7.2838  , p=0.0070  , df=1
parameter F test:         F=7.2930  , p=0.0072  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=4.1408  , p=0.0165  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=8.3645  , p=0.0153  , df=2
likelihood ratio test: chi2=8.2960  , p=0.0158  , df=2
parameter F test:         F=4.1408  , p=0.0165  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=7.2930  , p=0.0072  , df_denom=503, df_num=1
ssr based chi2 test: 

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=10.1876 , p=0.0015  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=10.2484 , p=0.0014  , df=1
likelihood ratio test: chi2=10.1460 , p=0.0014  , df=1
parameter F test:         F=10.1876 , p=0.0015  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=9.0568  , p=0.0001  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=18.2947 , p=0.0001  , df=2
likelihood ratio test: chi2=17.9711 , p=0.0001  , df=2
parameter F test:         F=9.0568  , p=0.0001  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=10.1876 , p=0.0015  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=10.2484 , p=0.0014  , df=1
likelihood ratio test: chi2=10.1460 , p=0.0014  , df=1
parameter F test:         F=10.1876 , p=0.0015  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=9.0568  , p=0.0001  ,

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.1716  , p=0.3107  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=2.3667  , p=0.3063  , df=2
likelihood ratio test: chi2=2.3612  , p=0.3071  , df=2
parameter F test:         F=1.1716  , p=0.3107  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=2.4315  , p=0.1196  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=2.4460  , p=0.1178  , df=1
likelihood ratio test: chi2=2.4401  , p=0.1183  , df=1
parameter F test:         F=2.4315  , p=0.1196  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.4974  , p=0.2247  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=3.0247  , p=0.2204  , df=2
likelihood ratio test: chi2=3.0156  , p=0.2214  , df=2
parameter F test:         F=1.4974  , p=0.2247  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=2.4315  , p=0.1196  ,

parameter F test:         F=3.7926  , p=0.0232  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.4777  , p=0.2247  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=1.4865  , p=0.2228  , df=1
likelihood ratio test: chi2=1.4843  , p=0.2231  , df=1
parameter F test:         F=1.4777  , p=0.2247  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.7214  , p=0.4866  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=1.4573  , p=0.4826  , df=2
likelihood ratio test: chi2=1.4552  , p=0.4831  , df=2
parameter F test:         F=0.7214  , p=0.4866  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.4777  , p=0.2247  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=1.4865  , p=0.2228  , df=1
likelihood ratio test: chi2=1.4843  , p=0.2231  , df=1
parameter F test:         F=1.4777  , p=0.2247  , df_denom=503, df_num=1

Granger Causality
nu

number of lags (no zero) 1
ssr based F test:         F=5.4580  , p=0.0199  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=5.4906  , p=0.0191  , df=1
likelihood ratio test: chi2=5.4610  , p=0.0194  , df=1
parameter F test:         F=5.4580  , p=0.0199  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=2.1817  , p=0.1139  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=4.4070  , p=0.1104  , df=2
likelihood ratio test: chi2=4.3879  , p=0.1115  , df=2
parameter F test:         F=2.1817  , p=0.1139  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=6.2677  , p=0.0126  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=6.3051  , p=0.0120  , df=1
likelihood ratio test: chi2=6.2662  , p=0.0123  , df=1
parameter F test:         F=6.2677  , p=0.0126  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=4.9450  , p=0.0075  , df_denom=500, df_

In [14]:
pd.set_option('display.max_columns',0) #replace n with the number of columns you want to see completely
pd.set_option('display.max_rows',0) #replace n with the number of rows you want to see completely

res_table.T.to_excel('中美汇率Top500-反向.xlsx')
res_table

,美国,口罩,武汉,湖北,开学,新冠,礼裙,原来,钟南山,肺炎,病例,本土,接种,出院,意大利,抗疫,特朗普,护士,世卫,疫苗,破防,感染,患者,无价,万例,时间,李兰娟,之姐,输入,推迟,女子,2021,复课,乘风破浪,号房,境外,上网,纽约州,纽约,方舱,触手可及,社死,宋仁宗,返校,男子,组织,累计,时代,非洲,医疗,援鄂,看待,重启,清平乐,复工,狗狗,自己,香港,历史,顶楼,...,东北,医务,emo,邮轮,UNINE,豆瓣,股市,明星,石家庄,不会,冬天,非裔,橘生,全球,想见,年级,义工,仙剑,54,症状,海市,各级,淡黄,顾佳,20,无能,受难,英国,挑战,汉克斯,航天员,空间站,马嘉祺,当家,全运会,不明,台湾,白宫,夜店,白鹿,主母,秦墩,半夜,华盛顿,好汉歌,淮南,吴宣仪,深夜,定金,变装,场面,忽悠,孝感,套装,柳絮,下药,贝宁,庆怜,最甜,王者
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,0,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1
1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1,1,1,1,0,1,0,1,1,0,1,1,1,1,1,0,1,1,0,0,0,1,1,1,0,0,0,0,1,1,0,0,1,0,1,1,1,1,1,1,1,0,1,0,0,1
